In [6]:
import json

with open('../pre_arrange/v2_mscoco_train2014_annotations.json', 'r') as f:
    json_data = json.load(f)

a=json.dumps(json_data, indent=4)

with open('../v2_mscoco_train2014_annotations_arranged_1.json', 'w') as file:
    file.write(a)

print('complete')

complete


#### 600000 더하기

In [ ]:
import json

input_file = '../v2_OpenEnded_mscoco_train2014_questions.json'

output_file = '../v2_OpenEnded_mscoco_train2014_questions_append.json'

with open(input_file, 'r') as f:
    data = json.load(f)

for question in data['questions']:
    question['image_id'] += 600000
    question['question_id'] += 600000000

with open(output_file, 'w') as f:
    json.dump(data, f, indent=4)

#### question 좌우반전

In [ ]:
import json
import re

with open('../v2_OpenEnded_mscoco_train2014_questions_arranged.json', 'r', encoding='utf-8') as f:
    questions_data = json.load(f)

def flip_right_left(question):
    # 먼저 "right"를 "LEFT_TEMP"로 바꿔서 중복 방지
    question = re.sub(r'\bright\b', 'LEFT_TEMP', question, flags=re.IGNORECASE)
    # "left"를 "right"로 바꿈
    question = re.sub(r'\bleft\b', 'right', question, flags=re.IGNORECASE)
    # "LEFT_TEMP"를 다시 "left"로 바꿈
    question = re.sub(r'LEFT_TEMP', 'left', question, flags=re.IGNORECASE)
    return question

for q in questions_data['questions']:
    q['question'] = flip_right_left(q['question'])

with open('../v2_OpenEnded_mscoco_train2014_questions_arranged_flip.json', 'w', encoding='utf-8') as f:
    json.dump(questions_data, f, ensure_ascii=False, indent=4)

print("Questions have been flipped and saved")

#### annotation 좌우반전

In [ ]:
# JSON 파일에 번호 매기기
import json

def add_set_numbers(input_file, output_file):
    # 입력 파일을 열어서 JSON 데이터를 읽음
    with open(input_file, 'r') as f:
        data = json.load(f)

    # 각 질문 세트에 번호 매기기
    for idx, question in enumerate(data["questions"], 1):
        question["set_number"] = idx

    # 수정된 데이터를 출력 파일에 저장
    with open(output_file, 'w') as f:
        json.dump(data, f, indent=2)

    print(f"새로운 JSON 파일이 '{output_file}' 이름으로 저장되었습니다.")

# 예시 사용법
input_file = './VQA/v2_OpenEnded_mscoco_val2014_questions_arranged.json'
output_file = './VQA/v2_OpenEnded_mscoco_val2014_questions_arranged_numbered.json'  # 수정된 데이터를 저장할 출력 파일 이름

add_set_numbers(input_file, output_file)

새로운 JSON 파일이 './VQA/v2_OpenEnded_mscoco_val2014_questions_arranged_numbered.json' 이름으로 저장되었습니다.


#### all answers 대답 빈도수 계산

In [5]:
import json
from collections import Counter

json_file_path = "../arranged/v2_mscoco_train2014_annotations.json"

with open(json_file_path, 'r') as f:
    data = json.load(f)

all_answers = []

for annotation in data['annotations']:
    for answer in annotation['answers']:
        all_answers.append(answer['answer'])

answer_count = Counter(all_answers)

with open("../all_answers_original.txt", 'w') as f:
    for answer, count in answer_count.items():
        f.write(f'"{answer}": {count}\n')

#### multiple choice answer 빈도수 계산

In [10]:
import json
from collections import Counter

json_file_path = '../annotations_integrated.json'
output_file_path = '../mc_answers_integrated.txt'

with open(json_file_path, 'r') as file:
    data = json.load(file)

answers = [annotation['multiple_choice_answer'] for annotation in data['annotations']]

answer_counts = Counter(answers)

with open(output_file_path, 'w') as file:
    for answer, count in answer_counts.items():
        file.write(f'"{answer}": {count}\n')

#### multiple choice answer 빈도수 기준 정렬

In [11]:
input_file_path = '../mc_answers_integrated.txt'
output_file_path = '../mc_answers_integrated_descending_order.txt'

with open(input_file_path, 'r') as file:
    lines = file.readlines()

answers = []
for line in lines:
    parts = line.strip().rsplit(': ', 1)
    if len(parts) == 2:
        answer, count = parts
        answers.append((answer.strip('"'), int(count)))

answers.sort(key=lambda x: x[1], reverse=True)

with open(output_file_path, 'w') as file:
    for answer, count in answers:
        file.write(f'"{answer}": {count}\n')

#### normalize 한 후 비교

In [13]:
import sys
sys.path.append('../../project')
from glossary import normalize_word

with open('../test.txt', 'r', encoding='utf-8') as file:
    data = file.readlines()

normalized_keys = set()

for line in data:
    key, _ = line.split(':', 1)
    normalized_key = normalize_word(key.strip())
    normalized_keys.add(normalized_key)

print(f"유니크한 값들의 갯수: {len(normalized_keys)}")

유니크한 값들의 갯수: 3003


#### txt 파일 비교

In [14]:
def find_unique_values(answer_file_path, test_file_path):
    with open(answer_file_path, 'r', encoding='utf-8') as f:
        answer_file_lines = f.readlines()
    
    answer_values = set()
    for line in answer_file_lines:
        if line.strip():
            line_dict = eval(line.strip())
            answer_values.add(line_dict["answer"])
    
    with open(test_file_path, 'r', encoding='utf-8') as f:
        test_file_lines = f.readlines()
    
    test_values = set()
    for line in test_file_lines:
        if line.strip():
            key, _ = line.split(':', 1)
            test_values.add(key.strip())
    
    answer_not_in_test = answer_values - test_values
    test_not_in_answer = test_values - answer_values
    
    print("answer2label_original.txt에 있지만 test.txt에는 없는 값:", answer_not_in_test)
    print("test.txt에 있지만 answer2label_original.txt에는 없는 값:", test_not_in_answer)

answer_file_path = '../answer2label/answer2label_original.txt'
test_file_path = '../test.txt'

find_unique_values(answer_file_path, test_file_path)

answer2label_original.txt에 있지만 test.txt에는 없는 값: {'', '88', 'afternoon', 'muffin', 'organic', 'privacy', 'jumped', 'easter', 'camera', 'bowling', 'leopard', 'rice', 'tiles', 'star alliance', 'in stands', 'brush', 'detroit', 'commercial', 's', 'worms', 'sunflower', 'tv', 'ceramic', 'mario', 'post', 'yellow and red', 'theater', 'on stove', 'dirt bike', '11:50', 'ribbon', 'floor', 'carnations', 'toilet paper', 'omelet', 'russia', 'db', 'aluminum', 'fishing', 'bank of america', 'dog and cat', 'below', 'graffiti', 'onion', 'run', 'washington monument', 'pot', '5:30', 'union station', 'harbor', 'sleep', 'huge', 'used', 'park', 'smile', 'crest', 'wheels', 'in cup', 'sprint', 'soccer field', 'candles', 'samsung', 'on horse', 'movie', '1 on left', 'blueberry', 'train car', 'on ground', 'healthy', 'heat', 'ivy', 'washing', '5:55', 'above toilet', 'exit', 'decoration', 'mickey mouse', 'ball', 'soccer ball', 'no hat', 'bull', 'stuffed animal', 'ski slope', 'ketchup and mustard', 'town', 'shopping',

#### json 파일 이어붙이기

In [7]:
import json

append_file = '../arranged/v2_mscoco_val2014_annotations.json'
original_file = '../arranged/v2_mscoco_train2014_annotations.json'
output_file = '../annotations_integrated.json'

with open(append_file, 'r') as f:
    append_data = json.load(f)

with open(original_file, 'r') as f:
    original_data = json.load(f)

original_data['annotations'].extend(append_data['annotations'])

with open(output_file, 'w') as f:
    json.dump(original_data, f, indent=4)

#### multiple choice answer만 JSON으로 저장하기

In [8]:
import json

input_file = '../annotations_integrated.json'
output_file = '../mc_answers_integrated.json'

with open(input_file, 'r') as f:
    data = json.load(f)

answers = []

for annotation in data['annotations']:
    entry = {
        "question_id": annotation['question_id'],
        "multiple_choice_answer": annotation['multiple_choice_answer']
    }
    answers.append(entry)

with open(output_file, 'w') as f:
    json.dump({"answers": answers}, f, indent=4)

print(f"총 {len(answers)}개의 답")

총 658111개의 답


#### 고유 대답 txt로 저장

In [4]:
import json

input_file = '../mc_answers.json'
output_file = '../answers_train.txt'

with open(input_file, 'r') as f:
    data = json.load(f)

unique_answers = set()

for entry in data['answers']:
    unique_answers.add(entry['multiple_choice_answer'])

with open(output_file, 'w') as f:
    for answer in sorted(unique_answers):
        f.write(answer + '\n')

print(f"총 {len(unique_answers)}개의 고유한 답변")

총 22633개의 고유한 답변


In [1]:
# 이미지 파일명 추가된 JSON 파일 만들기
import json

# JSON 파일 경로
input_file_path = './VQA/v2_OpenEnded_mscoco_val2014_questions_arranged_numbered.json'
output_file_path = './VQA/v2_OpenEnded_mscoco_val2014_questions_final.json'

# JSON 데이터 읽기
with open(input_file_path, 'r') as f:
    data = json.load(f)

# 각 질문 항목에 "image_filename" 추가
for question in data['questions']:
    image_id_str = f"{question['image_id']:012d}"
    question['image_filename'] = f"COCO_val2014_{image_id_str}.jpg"

# 새로운 JSON 파일로 저장
with open(output_file_path, 'w') as f:
    json.dump(data, f, indent=4)

print(f"New JSON file with 'image_filename' saved to {output_file_path}")

New JSON file with 'image_filename' saved to ./VQA/v2_OpenEnded_mscoco_val2014_questions_final.json


In [1]:
# annotation 파일 정답 묶음 만들기
import json
from collections import defaultdict

# JSON 파일을 읽기
with open('./VQA/annotations_5000.json', 'r') as file:
    data = json.load(file)

# 새로운 형식의 데이터 구조 생성
new_data = {
    "info": data["info"],
    "license": data["license"],
    "data_subtype": data["data_subtype"],
    "annotations": []
}

# annotation 변환 작업
for annotation in data["annotations"]:
    answer_counts = defaultdict(int)
    for answer in annotation["answers"]:
        key = f"{answer['answer']}_{answer['answer_confidence']}"
        answer_counts[key] += 1
    
    # 변환된 annotation 생성
    new_annotation = {
        "question_type": annotation["question_type"],
        "multiple_choice_answer": annotation["multiple_choice_answer"],
        "answers": answer_counts,
        "image_id": annotation["image_id"],
        "answer_type": annotation["answer_type"],
        "question_id": annotation["question_id"]
    }
    new_data["annotations"].append(new_annotation)

# 새로운 JSON 파일로 저장
with open('./VQA/new_annotations_5000.json', 'w') as file:
    json.dump(new_data, file, indent=2)

print("새로운 JSON 파일이 생성되었습니다.")

새로운 JSON 파일이 생성되었습니다.


In [1]:
# 어려운 질문 목록 뽑아내기
import json

# JSON 파일을 읽기
with open('./VQA/new_annotations_5000.json', 'r') as file:
    data = json.load(file)

# 일치하는 question_id를 저장할 리스트
matching_question_ids = []

# 각 annotation에 대해 조건 검사
for annotation in data['annotations']:
    multiple_choice_answer = annotation['multiple_choice_answer']
    answers = annotation['answers']

    # _yes로 끝나는 대답과 일치하는지 확인
    yes_answers = {key[:-4] for key in answers if key.endswith('_yes')}

    # _no나 _maybe로 끝나는 대답과 일치하는지 확인
    no_maybe_answers = {key[:-6] for key in answers if key.endswith('_no') or key.endswith('_maybe')}

    # multiple_choice_answer가 _yes 대답에는 없고, _no나 _maybe 대답에 있는 경우
    if multiple_choice_answer not in yes_answers and multiple_choice_answer in no_maybe_answers:
        matching_question_ids.append(annotation['question_id'])

# 결과 출력
print("일치하는 question_id:")
for question_id in matching_question_ids:
    print(question_id)

일치하는 question_id:
131108001
393267002
393277003
524361000
131207005
524450006
524450007
262323001
262334001
262376001
262386011
524536005
257004
524611002
524611007
262471002
328001
524621025
524621026
524637007
393569001
524642002
262509004
43753005
262531004
262531006
262576000
393674004
262605004
262605012
524775002
524799000
544002
262703002
393804003
262738000
187695000
636000
262800002
393874006
525024004
525024007
262896003
262896004
525077004
262985003
131938000
263011002
263014001
873003
885007
969001
394199000
985010
985015
132059000
394240002
263177001
132136001
132136002
394288002
394322004
423396001
1244000
132336007
525559001
525568002
525568006
525587000
1342010
1342019
1342038
1342062
1342074
525665010
525682002
525702008
394635001
263594001
394681002
263612001
263613000
525762004
132554005
1490002
1525004
132612003
263700003
1584002
1591002
1599001
525903002
132690000
263780009
22118000
22118004
263793001
196885004
1667003
263834002
525988020
526021003
526044000
132861

In [7]:
# 부분점수 주기
import json

# 파일 경로 목록
file_paths = [
    {
        'answers_paraphrased_path': './VQA/val2014_5000_answers_voted.json',
        'output_path': './VQA/val2014_5000_answers_voted_scores.json'
    }
]

# new_annotations 로드
with open('./VQA/new_annotations_5000.json', 'r') as f:
    new_annotations = json.load(f)

# 파일 경로마다 반복
for paths in file_paths:
    with open(paths['answers_paraphrased_path'], 'r') as f:
        val_answers_paraphrased = json.load(f)

    # "answers" 항목과 "answer" 비교 및 점수 부여
    answer_score_mapping = {}
    for annotation in new_annotations['annotations']:
        # image_id와 question_id를 키로 하여 매핑
        key = (annotation['image_id'], annotation['question_id'])
        answer_score_mapping[key] = annotation['answers']

    # "questions" 항목에 "score" 추가
    for question in val_answers_paraphrased['questions']:
        key = (question['image_id'], question['question_id'])
        answer = question['answer']
        scores = []

        # new_annotations에서 가져온 answer와 비교하여 점수 부여
        if key in answer_score_mapping:
            for ans_key, score in answer_score_mapping[key].items():
                if ans_key.split('_')[0] == answer:
                    scores.append(score)
            question['score'] = max(scores) if scores else 0
        else:
            question['score'] = 0

    # 결과 저장
    with open(paths['output_path'], 'w') as f:
        json.dump(val_answers_paraphrased, f, indent=4)

    print(f"새로운 JSON 파일이 {paths['output_path']}에 저장되었습니다.")

KeyError: 'image_id'

In [4]:
import json
import glob
import os

# 파일 경로
new_annotations_file = './VQA/new_annotations_5000.json'
paraphrased_answers_files_pattern = './VQA/val2014_5000_answers_paraphrased2_*.json'

# JSON 파일 읽기
with open(new_annotations_file, 'r') as f:
    new_annotations = json.load(f)

# question_id를 기준으로 매칭하여 데이터 업데이트
annotations_dict = {ann['question_id']: ann for ann in new_annotations['annotations']}

# 여러 paraphrased_answers_file을 처리
paraphrased_answers_files = glob.glob(paraphrased_answers_files_pattern)
for paraphrased_answers_file in paraphrased_answers_files:
    with open(paraphrased_answers_file, 'r') as f:
        paraphrased_answers = json.load(f)
    
    for question in paraphrased_answers['questions']:
        question_id = question['question_id']
        if question_id in annotations_dict:
            question['question_type'] = annotations_dict[question_id]['question_type']
            question['answer_type'] = annotations_dict[question_id]['answer_type']
    
    # 업데이트된 데이터 저장
    base_name = os.path.basename(paraphrased_answers_file)
    updated_file_path = os.path.join('./VQA', f'updated_{base_name}')
    with open(updated_file_path, 'w') as f:
        json.dump(paraphrased_answers, f, indent=4)